In [14]:

import pandas as pd
from Preprocessing.preprocessing import preprocessing_improved
from Models.LSTM.lightningLSTM import LightningLSTM
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from Preprocessing.dataclasses import StockPriceDataModule, StockDataset
import pytorch_lightning as pl

In [17]:
df = pd.read_csv('Data\Stock\StockBars\MSFT_Minute')
data_train, data_test = preprocessing_improved(df[:int(len(df)*0.1)], dif_all=True, lag=2, sequence_length=32)
data_module = StockPriceDataModule(data_train, data_test, batch_size=64)
data_module.setup()
"""
for item in data_module.train_dataloader():
    print(item.keys())
    print(item['sequence'].shape)
    print(item['label'].shape)
    break
"""

"\nfor item in data_module.train_dataloader():\n    print(item.keys())\n    print(item['sequence'].shape)\n    print(item['label'].shape)\n    break\n"

In [12]:
model = LightningLSTM(input_size=8, hidden_size = 10, num_layers = 1)
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoint",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = TensorBoardLogger("lightning_logs", name = "stock-price")

early_stopping_callback = EarlyStopping(monitor="val_loss", patience=2)

trainer = pl.Trainer(
    logger=logger,
    callbacks=[early_stopping_callback],
    max_epochs=1,
    accelerator='gpu'
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | model     | LSTM    | 811   
1 | criterion | MSELoss | 0     
--------------------------------------
811       Trainable params
0         Non-trainable params
811       Total params
0.003     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

RuntimeError: Early stopping conditioned on metric `val_loss` which is not available. Pass in or modify your `EarlyStopping` callback to use any of the following: `train_loss`